In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import re
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc
from nn_rag import Knowledge
from ds_core.handlers.abstract_handlers import ConnectorContract
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util


In [3]:
kn = Knowledge.from_memory()

In [4]:
text = ('You took too long. You are not easy to deal with. Payment Failure/Incorrect Payment. You provided '
        'me with incorrect information. Unhappy with delay. Unsuitable advice. You never answered my question. '
        'You did not understand my needs. I have been mis-sold. My details are not accurate. You have asked '
        'for too much information. You were not helpful. Payment not generated/received by customer. You did '
        'not keep me updated. Incorrect information given. The performance of my product was poor. No reply '
        'to customer contact. Requested documentation not issued. You did not explain the terms & conditions. '
        'Policy amendments not carried out. You did not explain the next steps/process to me. I cannot '
        'understand your letter/comms. Standard letter inappropriate. Customer payment processed incorrectly. '
        'All points not addressed. Could not understand the agent. Issue with terms and conditions. Misleading '
        'information. I can not use the customer portal. your customer portal is unhelpful')

## Milvus

In [5]:
from pymilvus import connections, db
from pymilvus import CollectionSchema, FieldSchema, DataType
from pymilvus import Collection, utility


In [6]:
conn = connections.connect(host="127.0.0.1", port=19530)
print(f"database list {db.list_database()}")

database list ['rai', 'scratchpad', 'default']


In [7]:
if "scratchpad" in db.list_database():
    db.drop_database("scratchpad")
database = db.create_database("scratchpad")

### Collections

In [8]:
collection_name = 'shorts'

if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

### Schema

In [9]:
fields = [
    FieldSchema(name="id", dtype=DataType.VARCHAR, auto_id=False, is_primary=True, max_length=64),
    FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=768)
]
# schema
schema = CollectionSchema(fields=fields)
# collection
collection = Collection(collection_name, schema)
# index
index_params = {
    "metric_type": "L2",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128}
}
collection.create_index("embeddings", index_params)

Status(code=0, message=)

### Partition

In [10]:
partition_name = 'docs'

if not collection.has_partition('docs'):
    collection.create_partition(partition_name=partition_name, description='')

{"name":"docs","collection_name":"shorts","description":""}